In [283]:
import pystac_client
import planetary_computer as pc
import pandas as pd
from datetime import datetime, timedelta
from odc.stac import stac_load
import xarray as xr
import numpy as np
import os
import multiprocessing as mp
import math
from pandarallel import pandarallel
from tqdm.contrib.concurrent import process_map  # or thread_map
import plotly.express as px


# from dotenv import load_dotenv
# load_dotenv()
# pc.settings.set_subscription_key(os.getenv('PC_SDK_SUBSCRIPTION_KEY'))

# Make data constants
SIZE = 'adaptative' # 'fixed'
FACTOR = 3
DEGREE = 0.0014589825157734703 # = ha_to_degree(2.622685) # Field size (ha) mean = 2.622685 (train + test)

In [382]:
def ha_to_degree(field_size): # Field_size (ha)
    ''' 
    1° ~= 111km
    1ha = 0.01km2
    then, side_size = sqrt(0.01 * field_size) (km)
    so, degree = side_size / 111 (°)
    '''
    side_size = math.sqrt(0.01 * field_size) 
    degree = side_size / 111
    return degree


def create_folders()->str:
    # os.makedirs('../../data/processed', exist_ok=True)
    if SIZE == 'fixed':
        degree = str(round(DEGREE, 5)).replace(".", "-")
        save_folder = f'../../data/processed/fixed_{degree}'
    elif SIZE == 'adaptative':
        save_folder = f'../../data/processed/adaptative_factor_{FACTOR}'
        
    os.makedirs(save_folder, exist_ok=True)
    return save_folder


dict_band_name = {
    'B05': 'rededge1',
    'B06': 'rededge2',
    'B07': 'rededge3',
    'B11': 'swir'
}

def band_to_name(band):
    if band == 'B05':
        band = 'rededge1'
    elif band == 'B06':
        band = 'rededge2'
    elif band == 'B07':
        band = 'rededge3'
    elif band == 'B11':
        band = 'swir'
    return band


def get_bbox(longitude, latitude, field_size):
    if SIZE == 'fixed':
        degree = DEGREE
    elif SIZE == 'adaptative':
        degree = ha_to_degree(field_size) * FACTOR
        
    min_longitude = longitude - degree / 2
    min_latitude = latitude - degree / 2
    max_longitude = longitude + degree / 2
    max_latitude = latitude + degree / 2 
    
    return (min_longitude, min_latitude, max_longitude, max_latitude)

def get_time_period(havest_date: str, history_days: int)->str:
    havest_datetime = datetime.strptime(havest_date, '%d-%m-%Y')
    sowing_datetime = havest_datetime - timedelta(days=history_days)
    return f'{sowing_datetime.strftime("%Y-%m-%d")}/{havest_datetime.strftime("%Y-%m-%d")}'


def get_data(bbox, time_period: str, bands: list[str], scale: float):
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace)
    search = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox, datetime=time_period)
    items = search.item_collection()
    data = stac_load(items, bands=bands, crs="EPSG:4326", resolution=scale, bbox=bbox)
    return data

def process_data(xds: xr.Dataset, row: pd.Series, history_dates:int)->xr.Dataset:
    # data = data.mean(dim=['latitude', 'longitude'], skipna=True)
    # data = data.to_dataframe()
    # data = data.sort_index(ascending=False).iloc[:history_dates]
    # data.index = data.index.round('D')
    # data.rename(columns=dict_band_name, inplace=True)
    # data.drop(columns=['SCL', 'spatial_ref'], inplace=True)
    # df = pd.DataFrame([row]*history_dates, index=data.index)
    # data = pd.concat([df, data], axis='columns')
    # data.reset_index(inplace=True)

    # xdf = data.copy(deep=True) 
    xds = xds.drop(['spatial_ref', 'SCL'])
    xds = xds.mean(dim=['latitude', 'longitude'], skipna=True)
    xds = xds.sortby('time', ascending=False)
    xds = xds.isel(time=slice(None, history_dates))
    xds['time'] = xds['time'].dt.strftime("%Y-%m-%d")
    xds['state_dev'] =  ('time', np.arange(history_dates)[::-1])
    xds = xds.swap_dims({'time': 'state_dev'})
    xds = xds.rename_vars(dict_band_name)
    df = pd.DataFrame([row]*history_dates, index=xds.indexes['state_dev'])
    xds = xds.merge(df.to_xarray())

    return xds


def save_data(row, history_days, history_dates, resolution):
    scale = resolution / 111320.0
    bands = ['red', 'green', 'blue', 'B05', 'B06', 'B07', 'nir', 'B11', 'SCL']
    
    longitude = row['Longitude']
    latitude = row['Latitude']
    field_size = float(row['Field size (ha)'])
    bbox = get_bbox(longitude, latitude, field_size)

    havest_date = row['Date of Harvest']
    time_period = get_time_period(havest_date, history_days)
    
    data = get_data(bbox, time_period, bands, scale)

    cloud_mask = \
        (data.SCL != 0) & \
        (data.SCL != 1) & \
        (data.SCL != 3) & \
        (data.SCL != 6) & \
        (data.SCL != 8) & \
        (data.SCL != 9) & \
        (data.SCL != 10)

    data_filter = data.copy(deep=True).where(cloud_mask)
    data = process_data(data, row, history_dates)
    data_filter = process_data(data_filter, row, history_dates)
    
    return data, data_filter

def save_data_app(index_row, history_days=130, history_dates=24, resolution=10):
    data, data_filter = save_data(index_row[1], history_days, history_dates, resolution)
    # data = save_data(index_row[1], history_days, history_dates, resolution)
    return data, data_filter

# def make_data(path, save_folder):
#     df = pd.read_csv(path)
#     print(f'\nRetrieve SAR data from {path.split("/")[-1]}...')
#     r = process_map(lambda _, row: save_data(row), df.iloc[:2].iterrows(), max_workers=8)
#     print(type(r))
#     # df['SAR data'] = df.apply(lambda row: save_data(row, df), axis=1)
#     # df = df.explode('SAR data')
#     # print(f'\nExplode SAR data from {path.split("/")[-1]}...')
#     # # df = df.parallel_apply(explode_sar, axis=1)
#     # df = df.apply(explode_sar, axis=1)
#     # df = df.drop(columns='SAR data')
#     # print(f'\nSave SAR data from {path.split("/")[-1]}...')
#     # df.to_csv(f'{save_folder}/{path.split("/")[-1]}', index=False)
#     # print(f'\nSAR data from {path.split("/")[-1]} saved!')

In [383]:
path = '../../data/raw/train.csv'
df = pd.read_csv(path)

# data, data_filter = save_data_app((df.iloc[0].index, df.iloc[0]))
data, data_filter = save_data_app((df.iloc[0].index, df.iloc[0]))
# data_2 = save_data_app((df.iloc[1].index, df.iloc[30]))
# data_3 = save_data_app((df.iloc[1].index, df.iloc[50]))

In [356]:
history_dates = 24

In [368]:
xdf = data.copy(deep=True)
xdf = xdf.drop(['spatial_ref', 'SCL'])
xdf = xdf.mean(dim=['latitude', 'longitude'], skipna=True)
xdf = xdf.sortby('time', ascending=False)
xdf = xdf.isel(time=slice(None, history_dates))
xdf['time'] = xdf['time'].dt.strftime("%Y-%m-%d")
xdf['state_dev'] =  ('time', np.arange(history_dates)[::-1])
xdf = xdf.swap_dims({'time': 'state_dev'})
xdf = xdf.rename_vars(dict_band_name)
df_t = pd.DataFrame([df.iloc[0]]*history_dates, index=xdf.indexes['state_dev'])
xdf = xdf.merge(df_t.to_xarray())

<xarray.Dataset>
Dimensions:                                         (state_dev: 24)
Coordinates:
    time                                            (state_dev) object '2022-...
  * state_dev                                       (state_dev) int64 23 ... 0
Data variables: (12/16)
    red                                             (state_dev) float32 nan ....
    green                                           (state_dev) float32 nan ....
    blue                                            (state_dev) float32 nan ....
    rededge1                                        (state_dev) float32 nan ....
    rededge2                                        (state_dev) float32 nan ....
    rededge3                                        (state_dev) float32 nan ....
    ...                                              ...
    Longitude                                       (state_dev) float64 105.2...
    Season(SA = Summer Autumn, WS = Winter Spring)  (state_dev) object 'SA' ....
    Rice Crop Intensity(D=Double, T=Triple)         (state_dev) object 'T' .....
    Date of Harvest                                 (state_dev) object '15-07...
    Field size (ha)                                 (state_dev) float64 3.4 ....
    Rice Yield (kg/ha)                              (state_dev) int64 5500 .....

In [344]:
xdf_2 = data_2.copy(deep=True)
xdf_2 = xdf_2.drop('spatial_ref')
xdf_2 = xdf_2.mean(dim=['latitude', 'longitude'], skipna=True)
xdf_2 = xdf_2.sortby('time')
xdf_2['time'] = xdf_2['time'].dt.strftime("%Y-%m-%d")
xdf_2['state_dev'] =  ('time', np.arange(xdf_2['time'].shape[0]))
xdf_2 = xdf_2.swap_dims({'time': 'state_dev'})

In [345]:
xdf_3 = data_3.copy(deep=True)
xdf_3 = xdf_3.drop('spatial_ref')
xdf_3 = xdf_3.mean(dim=['latitude', 'longitude'], skipna=True)
xdf_3 = xdf_3.sortby('time')
xdf_3['time'] = xdf_3['time'].dt.strftime("%Y-%m-%d")
xdf_3['state_dev'] =  ('time', np.arange(xdf_3['time'].shape[0]))
xdf_3 = xdf_3.swap_dims({'time': 'state_dev'})

In [348]:
xdf_concat = xr.concat([xdf, xdf_2, xdf_3], dim='ts_id')

In [350]:
xdf_concat.mean('ts_id', skipna=True)

<xarray.Dataset>
Dimensions:    (state_dev: 25)
Coordinates:
  * state_dev  (state_dev) int64 0 1 2 3 4 5 6 7 8 ... 17 18 19 20 21 22 23 24
Data variables:
    red        (state_dev) float32 1.5e+03 1.969e+03 2.415e+03 ... 1.743e+03 nan
    green      (state_dev) float32 1.754e+03 1.842e+03 ... 1.848e+03 nan
    blue       (state_dev) float32 1.489e+03 1.57e+03 ... 1.542e+03 nan
    B05        (state_dev) float32 2.14e+03 2.428e+03 ... 2.418e+03 nan
    B06        (state_dev) float32 3.835e+03 3.188e+03 ... 3.938e+03 nan
    B07        (state_dev) float32 4.643e+03 3.533e+03 ... 4.609e+03 nan
    nir        (state_dev) float32 4.67e+03 3.466e+03 ... 4.497e+03 nan
    B11        (state_dev) float32 2.599e+03 3.081e+03 ... 3.29e+03 nan
    SCL        (state_dev) float32 4.0 4.51 6.291 6.012 ... 4.366 7.0 4.159 nan

In [349]:
xdf_concat.fillna(xdf_concat.mean('ts_id', skipna=True))

<xarray.Dataset>
Dimensions:    (ts_id: 3, state_dev: 25)
Coordinates:
    time       (ts_id, state_dev) object '2022-03-11' ... '2022-08-03'
  * state_dev  (state_dev) int64 0 1 2 3 4 5 6 7 8 ... 17 18 19 20 21 22 23 24
Dimensions without coordinates: ts_id
Data variables:
    red        (ts_id, state_dev) float32 1.541e+03 1.969e+03 ... 2.178e+03 nan
    green      (ts_id, state_dev) float32 1.81e+03 1.842e+03 ... 2.14e+03 nan
    blue       (ts_id, state_dev) float32 1.466e+03 1.57e+03 ... 1.754e+03 nan
    B05        (ts_id, state_dev) float32 2.291e+03 2.428e+03 ... 2.977e+03 nan
    B06        (ts_id, state_dev) float32 3.94e+03 3.188e+03 ... 4.259e+03 nan
    B07        (ts_id, state_dev) float32 4.825e+03 3.533e+03 ... 4.862e+03 nan
    nir        (ts_id, state_dev) float32 4.854e+03 3.466e+03 ... 4.787e+03 nan
    B11        (ts_id, state_dev) float32 2.617e+03 3.081e+03 ... 3.931e+03 nan
    SCL        (ts_id, state_dev) float32 4.0 4.51 7.0 4.037 ... 7.0 4.299 nan

In [278]:
# index = pd.MultiIndex.from_arrays([xdf['time'].values, np.arange(0, xdf['time'].shape[0])], names=['time', 'state_dev'])
# xdf.assign_coords({'time': index})

ValueError: conflicting multi-index level name 'time' with dimension 'time'

In [136]:
fig = px.line(y=[(data_filter.nir - data_filter.red) / (data_filter.nir + data_filter.red), (data.nir - data.red) / (data.nir + data.red)], x=data.time)
fig.show()

Fill using mean of diff between data and filtered data adding to data to obtain filtered data 

In [132]:
test = (data[['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']] - data_filter[['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']]).mean()
df_test = data_filter.copy(deep=True)

df_test.loc[:, ['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']] = df_test[['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']].fillna(data[['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']] - test)

In [135]:
fig = px.line(y=[(df_test.nir - df_test.red) / (df_test.nir + df_test.red), (data.nir - data.red) / (data.nir + data.red)], x=df_test.time)
fig.show()

Fill using pandas interpolation

In [155]:
df_test = data_filter.copy(deep=True)

df_test.loc[:, ['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']] = df_test[['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']].interpolate(method="linear", axis="index", limit=4, limit_direction='both')

In [157]:
fig = px.line(y=[(df_test.nir - df_test.red) / (df_test.nir + df_test.red), (data.nir - data.red) / (data.nir + data.red)], x=df_test.time)
fig.show()

Smooth the data and fill nan on filter data

In [172]:
from scipy.signal import savgol_filter

df_test = data_filter.copy(deep=True)

arr = savgol_filter(data[['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']], axis=0, window_length=12, polyorder=4)
test = pd.DataFrame(arr, index=df_test.index, columns=['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir'])

df_test.loc[:, ['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']] = df_test[['red','green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']].fillna(test)

In [175]:
fig = px.line(y=[(df_test.nir - df_test.red) / (df_test.nir + df_test.red), (data.nir - data.red) / (data.nir + data.red)], x=df_test.time)
fig.show()

Use mean value of the same time index

In [215]:
def make_timeseries(df):
    df['timeserieindex'] = df['District'].copy(deep=True)
    for col in ['Latitude', 'Longitude', 'Season(SA = Summer Autumn, WS = Winter Spring)', 'Rice Crop Intensity(D=Double, T=Triple)', 'Date of Harvest', 'Field size (ha)', 'Rice Yield (kg/ha)']:
        df['timeserieindex'] += ' ' + df[col].astype('str')

    df['timeserieindex'] = pd.factorize(df['timeserieindex'].astype('str'))[0]
    return df

In [216]:
df_filter = pd.read_csv('../../data/processed/adaptative_factor_3/train_filter.csv')
df = pd.read_csv('../../data/processed/adaptative_factor_3/train.csv')


In [219]:
df = make_timeseries(df)
xarray.DataArray(df, coords=['place', 'index', 'bands'], )

<xarray.Dataset>
Dimensions:                                         (index: 13368, dim_0: 13368)
Coordinates:
  * index                                           (index) int64 0 1 ... 13367
    timeserieindex                                  (dim_0) int64 0 0 ... 556
  * dim_0                                           (dim_0) int64 0 1 ... 13367
Data variables: (12/17)
    time                                            (index) object '2022-07-1...
    District                                        (index) object 'Chau_Phu'...
    Latitude                                        (index) float64 10.51 ......
    Longitude                                       (index) float64 105.2 ......
    Season(SA = Summer Autumn, WS = Winter Spring)  (index) object 'SA' ... 'WS'
    Rice Crop Intensity(D=Double, T=Triple)         (index) object 'T' ... 'T'
    ...                                              ...
    blue                                            (index) float64 4.311e+03...
    rededge1                                        (index) float64 4.424e+03...
    rededge2                                        (index) float64 5.134e+03...
    rededge3                                        (index) float64 5.402e+03...
    nir                                             (index) float64 5.447e+03...
    swir                                            (index) float64 3.092e+03...